In [ ]:
import data
import matplotlib.pyplot as plt


loader_train, loader_test = data.get_data(32)
plt.imshow(loader_train.dataset[2][0].squeeze(), cmap="gray_r")

In [ ]:
from torch import nn


class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self._flatten = nn.Flatten()
        self._model = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 32),
            nn.Sigmoid(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        return self._model(self._flatten(x))


model = Model()
print(model)

In [ ]:
import torch
from torch import nn, optim
from tqdm.notebook import tqdm


loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())  

def train():
    with tqdm(desc="Batch", total=len(loader_train)) as pbatch:
        for (X, y) in loader_train:
            pred = model(X)
            loss = loss_function(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbatch.set_postfix(loss=loss.item())
            pbatch.update()

@torch.no_grad()
def test():
    test_loss = 0.0
    accuracy = 0.0
    for X, y in loader_test:
        pred = model(X)
        test_loss += loss_function(pred, y).item()
        accuracy += (torch.argmax(pred, dim=1) == torch.argmax(y, dim=1)).type(torch.float).sum()
    test_loss /= len(loader_test)
    accuracy /= len(loader_test.dataset)
    tqdm.write(f"Test -> Loss: {test_loss}, accuracy: {accuracy}")

In [ ]:
import torch
from tqdm.notebook import trange


torch.manual_seed(666)

for epoch in trange(1, 21, desc="Epoch"):
    train()
    test()